# VisualizeAUS: 20 Years of AUS Course Data

An in-depth data analysis of every course offered at the American University of Sharjah from 2005 to 2026, spanning 98 semesters, 73,418 course sections, 1,649 instructors, and 152,968 prerequisite/corequisite links.

**Data source:** [AUSCrawl](https://github.com/DeadPackets/AUSCrawl) — scraped from AUS Banner (Ellucian Banner)

In [ ]:
import sqlite3
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from collections import Counter, defaultdict
import re
import warnings
warnings.filterwarnings('ignore')

# Connect to database
conn = sqlite3.connect('aus_courses.db')
conn.row_factory = sqlite3.Row

# Global Plotly template
TEMPLATE = 'plotly_white'
COLORS = px.colors.qualitative.Set2
AUS_GOLD = '#C4972F'
AUS_DARK = '#1a1a2e'
AUS_BLUE = '#16213e'

print('Connected to database.')
query = "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence'"
tables = [r[0] for r in conn.execute(query).fetchall()]
print(f'Tables: {tables}')

---
## 1. University Growth Over 20 Years

How has AUS expanded its course offerings from 2005 to 2026?

In [ ]:
# Course count per semester
df_growth = pd.read_sql_query("""
    SELECT s.term_id, s.term_name,
           COUNT(*) as total_sections,
           COUNT(DISTINCT c.subject || c.course_number) as unique_courses,
           COUNT(DISTINCT c.instructor_name) as unique_instructors
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    GROUP BY c.term_id
    ORDER BY c.term_id
""", conn)

# Classify semester type
def classify_semester(name):
    name = name.lower()
    if 'fall' in name: return 'Fall'
    if 'spring' in name: return 'Spring'
    if 'wintermester' in name: return 'Wintermester'
    if 'summer ii' in name or 'summer 2' in name: return 'Summer II'
    if 'summer iii' in name or 'summer 3' in name: return 'Summer III'
    return 'Summer'

df_growth['semester_type'] = df_growth['term_name'].apply(classify_semester)
df_growth['year'] = df_growth['term_name'].str.extract(r'(\d{4})').astype(int)

# Filter to regular semesters (Fall + Spring) for trend line
df_regular = df_growth[df_growth['semester_type'].isin(['Fall', 'Spring'])].copy()

fig = go.Figure()

# All semesters as scatter
color_map = {'Fall': '#e74c3c', 'Spring': '#3498db', 'Summer': '#2ecc71', 
             'Summer II': '#27ae60', 'Summer III': '#1abc9c', 'Wintermester': '#9b59b6'}

for sem_type in ['Fall', 'Spring', 'Summer', 'Summer II', 'Wintermester']:
    mask = df_growth['semester_type'] == sem_type
    if mask.sum() == 0:
        continue
    fig.add_trace(go.Scatter(
        x=df_growth.loc[mask, 'term_name'],
        y=df_growth.loc[mask, 'total_sections'],
        mode='markers',
        name=sem_type,
        marker=dict(color=color_map.get(sem_type, '#95a5a6'), size=8),
        hovertemplate='%{x}<br>%{y} sections<extra></extra>'
    ))

# Trend line for regular semesters
z = np.polyfit(range(len(df_regular)), df_regular['total_sections'].values, 1)
p = np.poly1d(z)
fig.add_trace(go.Scatter(
    x=df_regular['term_name'],
    y=p(range(len(df_regular))),
    mode='lines',
    name=f'Trend (+{z[0]:.1f}/semester)',
    line=dict(color='rgba(0,0,0,0.3)', dash='dash', width=2)
))

fig.update_layout(
    template=TEMPLATE,
    title='AUS Course Sections Per Semester (2005–2026)',
    xaxis_title='Semester',
    yaxis_title='Number of Sections',
    height=500,
    xaxis=dict(tickangle=-45, dtick=4),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

growth_pct = (df_regular['total_sections'].iloc[-1] / df_regular['total_sections'].iloc[0] - 1) * 100
print(f"\nGrowth from {df_regular.iloc[0]['term_name']} to {df_regular.iloc[-1]['term_name']}: {growth_pct:.0f}%")
print(f"Average sections per regular semester: {df_regular['total_sections'].mean():.0f}")
print(f"Peak: {df_regular.loc[df_regular['total_sections'].idxmax(), 'term_name']} with {df_regular['total_sections'].max()} sections")

### 1.1 Unique Courses vs Total Sections Over Time

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=df_regular['term_name'], y=df_regular['total_sections'],
    name='Total Sections', mode='lines+markers',
    line=dict(color='#3498db', width=2), marker=dict(size=5)
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_regular['term_name'], y=df_regular['unique_courses'],
    name='Unique Courses', mode='lines+markers',
    line=dict(color='#e74c3c', width=2), marker=dict(size=5)
), secondary_y=True)

fig.update_layout(
    template=TEMPLATE,
    title='Total Sections vs Unique Courses (Regular Semesters)',
    height=450,
    xaxis=dict(tickangle=-45, dtick=4),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.update_yaxes(title_text='Total Sections', secondary_y=False)
fig.update_yaxes(title_text='Unique Courses', secondary_y=True)
fig.show()

df_regular = df_regular.copy()
df_regular['sections_per_course'] = df_regular['total_sections'] / df_regular['unique_courses']
print(f"Avg sections per unique course: {df_regular['sections_per_course'].mean():.2f}")
print(f"Trend: {df_regular['sections_per_course'].iloc[0]:.2f} → {df_regular['sections_per_course'].iloc[-1]:.2f} sections/course")

---
## 2. Subject Popularity and Evolution

Which departments dominate, and how has the academic landscape shifted?

In [ ]:
# Top 25 subjects by total sections
df_subjects = pd.read_sql_query("""
    SELECT c.subject, sub.long_name, COUNT(*) as total_sections,
           COUNT(DISTINCT c.term_id) as semesters_active,
           COUNT(DISTINCT c.instructor_name) as instructors
    FROM courses c
    JOIN subjects sub ON c.subject = sub.short_name
    GROUP BY c.subject
    ORDER BY total_sections DESC
""", conn)

top25 = df_subjects.head(25)

fig = px.bar(
    top25, x='subject', y='total_sections',
    color='total_sections',
    color_continuous_scale='Viridis',
    hover_data=['long_name', 'semesters_active', 'instructors'],
    labels={'total_sections': 'Total Sections', 'subject': 'Subject Code'},
    title='Top 25 Subjects by Total Course Sections (2005–2026)'
)
fig.update_layout(template=TEMPLATE, height=500, showlegend=False)
fig.update_coloraxes(showscale=False)
fig.show()

print(f"\nTotal subjects: {len(df_subjects)}")
print(f"Top 10 subjects account for {df_subjects.head(10)['total_sections'].sum() / df_subjects['total_sections'].sum() * 100:.1f}% of all sections")

In [ ]:
# Subject evolution over time — animated
df_subj_time = pd.read_sql_query("""
    SELECT c.subject, s.term_name, s.term_id, COUNT(*) as sections
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%'
    GROUP BY c.subject, s.term_id
    ORDER BY s.term_id
""", conn)

# Top 15 subjects for readability
top15_subjects = df_subjects.head(15)['subject'].tolist()
df_top15 = df_subj_time[df_subj_time['subject'].isin(top15_subjects)].copy()

fig = px.line(
    df_top15, x='term_name', y='sections', color='subject',
    title='Top 15 Subjects: Section Count Over Time (Regular Semesters)',
    labels={'sections': 'Sections', 'term_name': 'Semester'},
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_layout(
    template=TEMPLATE, height=550,
    xaxis=dict(tickangle=-45, dtick=4),
    legend=dict(title='Subject')
)
fig.show()

In [ ]:
# Subject heatmap — sections per subject per year
df_heatmap = pd.read_sql_query("""
    SELECT c.subject, 
           CAST(SUBSTR(s.term_name, -4) AS INTEGER) as year,
           COUNT(*) as sections
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    GROUP BY c.subject, year
""", conn)

# Pivot for heatmap — top 30 subjects
top30 = df_subjects.head(30)['subject'].tolist()
df_hm = df_heatmap[df_heatmap['subject'].isin(top30)].pivot_table(
    index='subject', columns='year', values='sections', fill_value=0
)
# Sort by total
df_hm = df_hm.loc[df_hm.sum(axis=1).sort_values(ascending=True).index]

fig = px.imshow(
    df_hm, aspect='auto',
    color_continuous_scale='YlOrRd',
    title='Course Sections Heatmap: Top 30 Subjects by Year',
    labels={'x': 'Year', 'y': 'Subject', 'color': 'Sections'}
)
fig.update_layout(template=TEMPLATE, height=700)
fig.show()

### 2.1 New Subjects and Programs Timeline

When did new subjects first appear?

In [ ]:
df_first_seen = pd.read_sql_query("""
    SELECT short_name as subject, long_name, first_seen
    FROM subjects
    ORDER BY first_seen
""", conn)

# Count new subjects per semester
df_new_per_sem = df_first_seen.groupby('first_seen').size().reset_index(name='new_subjects')

# Also get level milestones
df_levels = pd.read_sql_query("SELECT level, first_seen FROM levels ORDER BY first_seen", conn)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_new_per_sem['first_seen'], y=df_new_per_sem['new_subjects'],
    name='New Subjects',
    marker_color=AUS_GOLD,
    hovertemplate='%{x}<br>%{y} new subjects<extra></extra>'
))

# Annotate level milestones
for _, row in df_levels.iterrows():
    if row['first_seen'] and 'Undergraduate' not in row['level']:
        fig.add_vline(x=row['first_seen'], line_dash='dot', line_color='rgba(0,0,0,0.3)')
        fig.add_annotation(
            x=row['first_seen'], y=df_new_per_sem['new_subjects'].max(),
            text=row['level'], showarrow=False,
            textangle=-45, font=dict(size=9, color='gray'),
            yshift=10
        )

fig.update_layout(
    template=TEMPLATE,
    title='New Subjects Introduced Per Semester + Academic Level Milestones',
    xaxis_title='Semester (first_seen)',
    yaxis_title='New Subjects',
    height=450,
    xaxis=dict(tickangle=-45)
)
fig.show()

# Show recent additions
print("\nSubjects introduced after 2015:")
recent = df_first_seen[df_first_seen['first_seen'] > 'Fall 2015']
for _, row in recent.iterrows():
    print(f"  {row['subject']:6s} {row['long_name']:45s} ({row['first_seen']})")

---
## 3. Instructor Analysis

Who are the most prolific instructors? How has the teaching workforce changed?

In [ ]:
# Top 30 instructors (excluding TBA)
df_instructors = pd.read_sql_query("""
    SELECT instructor_name, COUNT(*) as total_sections,
           COUNT(DISTINCT subject) as subjects_taught,
           COUNT(DISTINCT term_id) as semesters_active,
           MIN(term_id) as first_term, MAX(term_id) as last_term
    FROM courses
    WHERE instructor_name != '' AND instructor_name != 'TBA'
    GROUP BY instructor_name
    ORDER BY total_sections DESC
""", conn)

top30_inst = df_instructors.head(30)

fig = px.bar(
    top30_inst.iloc[::-1], x='total_sections', y='instructor_name',
    orientation='h',
    color='semesters_active',
    color_continuous_scale='Blues',
    hover_data=['subjects_taught', 'semesters_active'],
    title='Top 30 Instructors by Total Sections Taught',
    labels={'total_sections': 'Total Sections', 'instructor_name': '',
            'semesters_active': 'Semesters Active'}
)
fig.update_layout(template=TEMPLATE, height=700)
fig.show()

print(f"Total unique instructors (excl. TBA): {len(df_instructors)}")
print(f"Average sections per instructor: {df_instructors['total_sections'].mean():.1f}")
print(f"Median sections per instructor: {df_instructors['total_sections'].median():.0f}")
print(f"Instructors with 100+ sections: {(df_instructors['total_sections'] >= 100).sum()}")
print(f"Instructors with only 1 section: {(df_instructors['total_sections'] == 1).sum()}")

In [ ]:
# Instructor tenure distribution
fig = px.histogram(
    df_instructors, x='semesters_active', nbins=40,
    title='Instructor Tenure Distribution (Number of Semesters Active)',
    labels={'semesters_active': 'Semesters Active', 'count': 'Instructors'},
    color_discrete_sequence=[AUS_GOLD]
)
fig.update_layout(template=TEMPLATE, height=400)
fig.show()

print(f"Instructors active 20+ semesters (10+ years): {(df_instructors['semesters_active'] >= 20).sum()}")
print(f"Instructors active only 1 semester: {(df_instructors['semesters_active'] == 1).sum()}")

In [ ]:
# Active instructors per semester
df_active_inst = pd.read_sql_query("""
    SELECT s.term_name, s.term_id,
           COUNT(DISTINCT c.instructor_name) as active_instructors
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE c.instructor_name != '' AND c.instructor_name != 'TBA'
    AND (s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%')
    GROUP BY c.term_id
    ORDER BY c.term_id
""", conn)

fig = px.area(
    df_active_inst, x='term_name', y='active_instructors',
    title='Active Instructors Per Regular Semester',
    labels={'active_instructors': 'Active Instructors', 'term_name': 'Semester'},
    color_discrete_sequence=[AUS_GOLD]
)
fig.update_layout(template=TEMPLATE, height=400, xaxis=dict(tickangle=-45, dtick=4))
fig.show()

print(f"Faculty growth: {df_active_inst.iloc[0]['active_instructors']} → {df_active_inst.iloc[-1]['active_instructors']} instructors")

In [ ]:
# TBA rate over time — what % of sections have no assigned instructor?
df_tba = pd.read_sql_query("""
    SELECT s.term_name, s.term_id,
           SUM(CASE WHEN c.instructor_name = '' OR c.instructor_name = 'TBA' THEN 1 ELSE 0 END) as tba_count,
           COUNT(*) as total,
           ROUND(100.0 * SUM(CASE WHEN c.instructor_name = '' OR c.instructor_name = 'TBA' THEN 1 ELSE 0 END) / COUNT(*), 1) as tba_pct
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%'
    GROUP BY c.term_id
    ORDER BY c.term_id
""", conn)

fig = px.bar(
    df_tba, x='term_name', y='tba_pct',
    title='Percentage of Sections with No Assigned Instructor (TBA)',
    labels={'tba_pct': 'TBA %', 'term_name': 'Semester'},
    color='tba_pct', color_continuous_scale='RdYlGn_r'
)
fig.update_layout(template=TEMPLATE, height=400, xaxis=dict(tickangle=-45, dtick=4))
fig.update_coloraxes(showscale=False)
fig.show()

print(f"Average TBA rate: {df_tba['tba_pct'].mean():.1f}%")
print(f"Latest semester TBA rate: {df_tba.iloc[-1]['tba_pct']}%")

---
## 4. Schedule Analysis

When do AUS students have their classes? What does the weekly schedule look like?

In [ ]:
# Day-time heatmap
df_schedule = pd.read_sql_query("""
    SELECT days, start_time, COUNT(*) as count
    FROM courses
    WHERE days != '' AND start_time != '' AND start_time != '12:00 am'
    GROUP BY days, start_time
""", conn)

# Parse time for sorting
def parse_time(t):
    try:
        parts = t.strip().split()
        h, m = map(int, parts[0].split(':'))
        if parts[1].lower() == 'pm' and h != 12:
            h += 12
        if parts[1].lower() == 'am' and h == 12:
            h = 0
        return h * 60 + m
    except:
        return 9999

# Expand day patterns into individual days
day_map = {'M': 'Mon', 'T': 'Tue', 'W': 'Wed', 'R': 'Thu', 'U': 'Sun', 'S': 'Sat'}
day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Sat']

rows = []
for _, r in df_schedule.iterrows():
    for ch in r['days']:
        if ch in day_map:
            rows.append({'day': day_map[ch], 'start_time': r['start_time'], 
                        'count': r['count'], 'time_mins': parse_time(r['start_time'])})

df_expanded = pd.DataFrame(rows)
df_pivot = df_expanded.pivot_table(index='start_time', columns='day', values='count', 
                                    aggfunc='sum', fill_value=0)

# Sort by time
df_pivot['time_mins'] = df_pivot.index.map(parse_time)
df_pivot = df_pivot.sort_values('time_mins')
df_pivot = df_pivot.drop('time_mins', axis=1)
df_pivot = df_pivot[[d for d in day_order if d in df_pivot.columns]]

fig = px.imshow(
    df_pivot.values,
    x=df_pivot.columns.tolist(),
    y=df_pivot.index.tolist(),
    color_continuous_scale='YlOrRd',
    title='Schedule Heatmap: When Are AUS Courses Held?',
    labels={'x': 'Day', 'y': 'Start Time', 'color': 'Sections'},
    aspect='auto'
)
fig.update_layout(template=TEMPLATE, height=600)
fig.show()

print("\nBusiest slots (day × time):")
top_slots = df_expanded.groupby(['day', 'start_time'])['count'].sum().sort_values(ascending=False).head(10)
for (day, time), count in top_slots.items():
    print(f"  {day} {time}: {count} sections")

In [ ]:
# Day pattern popularity
df_days = pd.read_sql_query("""
    SELECT days, COUNT(*) as count
    FROM courses
    WHERE days != ''
    GROUP BY days
    ORDER BY count DESC
    LIMIT 10
""", conn)

day_labels = {
    'MW': 'Mon/Wed', 'TRU': 'Tue/Thu/Sun', 'TR': 'Tue/Thu',
    'M': 'Mon', 'W': 'Wed', 'T': 'Tue', 'MTWRU': 'Mon–Thu+Sun',
    'TU': 'Tue/Sun', 'R': 'Thu', 'U': 'Sun', 'MWS': 'Mon/Wed/Sat'
}
df_days['label'] = df_days['days'].map(lambda d: day_labels.get(d, d))

fig = px.pie(
    df_days, values='count', names='label',
    title='Most Common Day Patterns (All Semesters)',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(template=TEMPLATE, height=450)
fig.show()

print(f"MW (Mon/Wed 75min blocks): {df_days.iloc[0]['count']} sections ({df_days.iloc[0]['count']/df_days['count'].sum()*100:.1f}%)")
print(f"TRU (Tue/Thu/Sun 50min blocks): {df_days.iloc[1]['count']} sections ({df_days.iloc[1]['count']/df_days['count'].sum()*100:.1f}%)")

In [ ]:
# Classroom utilization — top buildings
df_rooms = pd.read_sql_query("""
    SELECT classroom as location, COUNT(*) as count
    FROM courses
    WHERE classroom != '' AND classroom != 'TBA'
    GROUP BY classroom
    ORDER BY count DESC
""", conn)

# Extract building name
def get_building(loc):
    loc = str(loc)
    if 'New Academic' in loc: return 'New Academic Bldg 1'
    if 'Language' in loc: return 'Language Building'
    if 'Engineering Building Right' in loc or 'EB2' in loc: return 'Engineering Bldg Right (EB2)'
    if 'Engineering Building Left' in loc or 'EB1' in loc: return 'Engineering Bldg Left (EB1)'
    if 'Main Building' in loc or 'MB' in loc: return 'Main Building'
    if 'Science' in loc: return 'Science Building'
    if 'Architecture' in loc or 'AB' in loc: return 'Architecture Building'
    if 'Sports' in loc or 'Gym' in loc: return 'Sports Complex'
    if 'Library' in loc: return 'Library'
    return 'Other'

df_rooms['building'] = df_rooms['location'].apply(get_building)
df_buildings = df_rooms.groupby('building')['count'].sum().sort_values(ascending=False).reset_index()
df_buildings = df_buildings[df_buildings['building'] != 'Other']

fig = px.bar(
    df_buildings, x='building', y='count',
    title='Course Sections by Building (All Semesters)',
    labels={'count': 'Total Sections', 'building': ''},
    color='count', color_continuous_scale='Teal'
)
fig.update_layout(template=TEMPLATE, height=400)
fig.update_coloraxes(showscale=False)
fig.show()

---
## 5. Prerequisite Dependency Graph

AUS courses form a complex web of prerequisites and corequisites. Let's visualize the dependency structure.

In [ ]:
# Build dependency graph
df_deps = pd.read_sql_query("""
    SELECT DISTINCT
        c.subject as course_subject, c.course_number as course_num,
        d.subject as dep_subject, d.course_number as dep_num,
        d.dep_type, d.minimum_grade
    FROM course_dependencies d
    JOIN courses c ON c.crn = d.crn AND c.term_id = d.term_id
    WHERE d.dep_type = 'prerequisite'
""", conn)

# Build directed graph: prerequisite → course
G = nx.DiGraph()
for _, row in df_deps.iterrows():
    src = f"{row['dep_subject']} {row['dep_num']}"
    dst = f"{row['course_subject']} {row['course_num']}"
    if src != dst:  # avoid self-loops
        G.add_edge(src, dst, grade=row['minimum_grade'])

print(f"Dependency graph: {G.number_of_nodes()} courses, {G.number_of_edges()} prerequisite edges")
print(f"Courses with no prerequisites (roots): {sum(1 for n in G.nodes() if G.in_degree(n) == 0)}")
print(f"Courses with no dependents (leaves): {sum(1 for n in G.nodes() if G.out_degree(n) == 0)}")

In [ ]:
# Find longest prerequisite chains
def find_longest_paths(G):
    """Find the longest prerequisite chain for each course."""
    longest = {}
    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        if not preds:
            longest[node] = [node]
        else:
            best = max((longest.get(p, [p]) for p in preds), key=len)
            longest[node] = best + [node]
    return longest

try:
    paths = find_longest_paths(G)
    sorted_paths = sorted(paths.items(), key=lambda x: len(x[1]), reverse=True)
    
    print("Top 15 longest prerequisite chains:")
    print("=" * 80)
    for course, path in sorted_paths[:15]:
        print(f"  [{len(path)} steps] {' → '.join(path)}")
except nx.NetworkXUnfeasible:
    print("Graph has cycles — computing longest paths in DAG subcomponents...")
    # Remove cycles
    cycles = list(nx.simple_cycles(G))
    print(f"Found {len(cycles)} cycles")
    for cycle in cycles[:5]:
        print(f"  Cycle: {' → '.join(cycle)}")

In [ ]:
# Most connected courses (highest in-degree + out-degree)
degree_data = []
for node in G.nodes():
    degree_data.append({
        'course': node,
        'prerequisites': G.in_degree(node),
        'is_prereq_for': G.out_degree(node),
        'total_connections': G.in_degree(node) + G.out_degree(node)
    })

df_degree = pd.DataFrame(degree_data).sort_values('total_connections', ascending=False)

# Top 25 most connected courses
top25_deg = df_degree.head(25)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=top25_deg['course'], x=top25_deg['is_prereq_for'],
    name='Is prerequisite for', orientation='h',
    marker_color='#e74c3c'
))
fig.add_trace(go.Bar(
    y=top25_deg['course'], x=top25_deg['prerequisites'],
    name='Has prerequisites', orientation='h',
    marker_color='#3498db'
))

fig.update_layout(
    template=TEMPLATE,
    title='Most Connected Courses in the Prerequisite Graph',
    xaxis_title='Number of Connections',
    barmode='stack',
    height=650,
    yaxis=dict(autorange='reversed'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

In [ ]:
# Interactive prerequisite network for a specific department (COE as example)
dept = 'COE'

# Get all COE courses and their dependencies
coe_courses = {n for n in G.nodes() if n.startswith(f'{dept} ')}
# Include direct prerequisites (even from other departments)
coe_extended = set(coe_courses)
for c in coe_courses:
    coe_extended.update(G.predecessors(c))
    coe_extended.update(G.successors(c))

subG = G.subgraph(coe_extended)

# Layout
pos = nx.spring_layout(subG, k=2, iterations=50, seed=42)

# Edges
edge_x, edge_y = [], []
for u, v in subG.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=0.5, color='#888'),
    hoverinfo='none'
)

# Nodes
node_x = [pos[n][0] for n in subG.nodes()]
node_y = [pos[n][1] for n in subG.nodes()]
node_text = list(subG.nodes())
node_color = ['#e74c3c' if n.startswith(f'{dept} ') else '#3498db' for n in subG.nodes()]
node_size = [10 + subG.degree(n) * 2 for n in subG.nodes()]

node_trace = go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    hoverinfo='text',
    text=node_text,
    textposition='top center',
    textfont=dict(size=8),
    marker=dict(color=node_color, size=node_size, line=dict(width=1, color='white'))
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    template=TEMPLATE,
    title=f'{dept} Prerequisite Network (red = {dept}, blue = other departments)',
    showlegend=False,
    height=700,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.show()

In [ ]:
# Prerequisite complexity by department
df_dept_deps = pd.read_sql_query("""
    SELECT c.subject,
           COUNT(DISTINCT d.id) as total_dependencies,
           COUNT(DISTINCT c.subject || c.course_number) as unique_courses,
           ROUND(1.0 * COUNT(DISTINCT d.id) / COUNT(DISTINCT c.subject || c.course_number), 1) as deps_per_course
    FROM course_dependencies d
    JOIN courses c ON c.crn = d.crn AND c.term_id = d.term_id
    WHERE d.dep_type = 'prerequisite'
    GROUP BY c.subject
    HAVING unique_courses >= 5
    ORDER BY deps_per_course DESC
""", conn)

fig = px.bar(
    df_dept_deps.head(25), x='subject', y='deps_per_course',
    color='deps_per_course', color_continuous_scale='Reds',
    hover_data=['total_dependencies', 'unique_courses'],
    title='Average Prerequisites Per Course by Department (Most Complex First)',
    labels={'deps_per_course': 'Avg Prerequisites/Course', 'subject': 'Subject'}
)
fig.update_layout(template=TEMPLATE, height=450)
fig.update_coloraxes(showscale=False)
fig.show()

---
## 6. Grade Requirements and Academic Rigor

What minimum grades do courses require? How strict are different departments?

In [ ]:
# Grade requirement distribution
df_grades = pd.read_sql_query("""
    SELECT minimum_grade, COUNT(*) as count
    FROM course_dependencies
    WHERE minimum_grade != ''
    GROUP BY minimum_grade
    ORDER BY count DESC
""", conn)

grade_order = ['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'P']
df_grades['grade_rank'] = df_grades['minimum_grade'].map({g: i for i, g in enumerate(grade_order)})
df_grades = df_grades.sort_values('grade_rank')

fig = px.bar(
    df_grades, x='minimum_grade', y='count',
    title='Minimum Grade Requirements Across All Prerequisites',
    labels={'minimum_grade': 'Minimum Grade', 'count': 'Number of Dependencies'},
    color='count', color_continuous_scale='RdYlGn_r',
    text='count'
)
fig.update_traces(textposition='outside')
fig.update_layout(template=TEMPLATE, height=400)
fig.update_coloraxes(showscale=False)
fig.show()

total = df_grades['count'].sum()
c_minus = df_grades[df_grades['minimum_grade'] == 'C-']['count'].values[0]
print(f"C- accounts for {c_minus/total*100:.1f}% of all grade requirements")
higher_than_c = df_grades[df_grades['minimum_grade'].isin(['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C'])]['count'].sum()
print(f"Requirements stricter than C-: {higher_than_c} ({higher_than_c/total*100:.1f}%)")

In [ ]:
# Which departments require higher grades?
df_strict = pd.read_sql_query("""
    SELECT c.subject,
           SUM(CASE WHEN d.minimum_grade IN ('A', 'A-') THEN 1 ELSE 0 END) as grade_A,
           SUM(CASE WHEN d.minimum_grade IN ('B+', 'B', 'B-') THEN 1 ELSE 0 END) as grade_B,
           SUM(CASE WHEN d.minimum_grade IN ('C+', 'C') THEN 1 ELSE 0 END) as grade_C,
           SUM(CASE WHEN d.minimum_grade = 'C-' THEN 1 ELSE 0 END) as grade_C_minus,
           SUM(CASE WHEN d.minimum_grade IN ('D+', 'D', 'D-') THEN 1 ELSE 0 END) as grade_D,
           COUNT(*) as total
    FROM course_dependencies d
    JOIN courses c ON c.crn = d.crn AND c.term_id = d.term_id
    WHERE d.minimum_grade != ''
    GROUP BY c.subject
    HAVING total >= 50
    ORDER BY 1.0 * (grade_A + grade_B) / total DESC
""", conn)

df_strict['strict_pct'] = (df_strict['grade_A'] + df_strict['grade_B']) / df_strict['total'] * 100

fig = go.Figure()
for grade_col, color, label in [
    ('grade_A', '#e74c3c', 'A/A-'),
    ('grade_B', '#f39c12', 'B range'),
    ('grade_C', '#3498db', 'C/C+'),
    ('grade_C_minus', '#2ecc71', 'C-'),
    ('grade_D', '#95a5a6', 'D range')
]:
    fig.add_trace(go.Bar(
        x=df_strict['subject'], y=df_strict[grade_col],
        name=label, marker_color=color
    ))

fig.update_layout(
    template=TEMPLATE,
    title='Grade Requirement Strictness by Department',
    barmode='stack',
    height=500,
    xaxis_title='Subject',
    yaxis_title='Number of Dependencies',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

---
## 7. Catalog Analysis: Credit Hours and Course Structure

In [ ]:
# Credit hours distribution
df_catalog = pd.read_sql_query("""
    SELECT subject, course_number, description, credit_hours, 
           lecture_hours, lab_hours, department
    FROM catalog
    WHERE credit_hours > 0 AND credit_hours < 30
""", conn)

fig = px.histogram(
    df_catalog, x='credit_hours', nbins=20,
    title='Credit Hours Distribution Across All Courses',
    labels={'credit_hours': 'Credit Hours', 'count': 'Number of Courses'},
    color_discrete_sequence=[AUS_GOLD]
)
fig.update_layout(template=TEMPLATE, height=400)
fig.show()

print(f"3-credit courses: {(df_catalog['credit_hours'] == 3).sum()} ({(df_catalog['credit_hours'] == 3).sum()/len(df_catalog)*100:.1f}%)")
print(f"Courses with lab hours: {(df_catalog['lab_hours'] > 0).sum()}")
print(f"Average credit hours: {df_catalog['credit_hours'].mean():.2f}")

In [ ]:
# Lecture vs Lab hours by department
df_dept_hours = df_catalog.groupby('department').agg({
    'lecture_hours': 'mean',
    'lab_hours': 'mean',
    'credit_hours': ['mean', 'count']
}).reset_index()
df_dept_hours.columns = ['department', 'avg_lecture', 'avg_lab', 'avg_credits', 'course_count']
df_dept_hours = df_dept_hours[df_dept_hours['course_count'] >= 10].sort_values('avg_lab', ascending=False)

# Clean department names
df_dept_hours['dept_short'] = df_dept_hours['department'].str.replace(' Department', '').str.replace(' (n.a.)', '', regex=False)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_dept_hours['dept_short'], x=df_dept_hours['avg_lecture'],
    name='Lecture Hours', orientation='h', marker_color='#3498db'
))
fig.add_trace(go.Bar(
    y=df_dept_hours['dept_short'], x=df_dept_hours['avg_lab'],
    name='Lab Hours', orientation='h', marker_color='#e74c3c'
))

fig.update_layout(
    template=TEMPLATE,
    title='Average Lecture vs Lab Hours by Department',
    barmode='stack',
    height=600,
    xaxis_title='Hours',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

---
## 8. Enrollment and Capacity Analysis

In [ ]:
# Seat availability trends over time
df_seats = pd.read_sql_query("""
    SELECT s.term_name, s.term_id,
           SUM(CASE WHEN c.seats_available = 1 THEN 1 ELSE 0 END) as available,
           SUM(CASE WHEN c.seats_available = 0 THEN 1 ELSE 0 END) as full_sections,
           COUNT(*) as total,
           ROUND(100.0 * SUM(CASE WHEN c.seats_available = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as full_pct
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%'
    GROUP BY c.term_id
    ORDER BY c.term_id
""", conn)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_seats['term_name'], y=df_seats['available'],
    name='Available', marker_color='#2ecc71'
))
fig.add_trace(go.Bar(
    x=df_seats['term_name'], y=df_seats['full_sections'],
    name='Full', marker_color='#e74c3c'
))

fig.update_layout(
    template=TEMPLATE,
    title='Section Availability: Available vs Full Sections Per Semester',
    barmode='stack',
    height=450,
    xaxis=dict(tickangle=-45, dtick=4),
    yaxis_title='Number of Sections',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()

print(f"Average % full: {df_seats['full_pct'].mean():.1f}%")
print(f"Most congested semester: {df_seats.loc[df_seats['full_pct'].idxmax(), 'term_name']} ({df_seats['full_pct'].max()}% full)")

In [ ]:
# Most frequently full subjects
df_subj_full = pd.read_sql_query("""
    SELECT subject,
           SUM(CASE WHEN seats_available = 0 THEN 1 ELSE 0 END) as full_count,
           COUNT(*) as total,
           ROUND(100.0 * SUM(CASE WHEN seats_available = 0 THEN 1 ELSE 0 END) / COUNT(*), 1) as full_pct
    FROM courses
    GROUP BY subject
    HAVING total >= 50
    ORDER BY full_pct DESC
""", conn)

fig = px.bar(
    df_subj_full.head(25), x='subject', y='full_pct',
    color='full_pct', color_continuous_scale='RdYlGn_r',
    hover_data=['full_count', 'total'],
    title='Subject Fill Rate: % of Sections That Were Full (Top 25)',
    labels={'full_pct': '% Full', 'subject': 'Subject'}
)
fig.update_layout(template=TEMPLATE, height=450)
fig.update_coloraxes(showscale=False)
fig.show()

---
## 9. Fee Analysis

In [ ]:
# Parse fees from JSON
df_fees_raw = pd.read_sql_query("""
    SELECT sd.crn, sd.term_id, sd.fees, c.subject, c.title,
           s.term_name
    FROM section_details sd
    JOIN courses c ON c.crn = sd.crn AND c.term_id = sd.term_id
    JOIN semesters s ON s.term_id = sd.term_id
    WHERE sd.fees != '' AND sd.fees != '[]'
""", conn)

fee_records = []
for _, row in df_fees_raw.iterrows():
    try:
        fees = json.loads(row['fees'])
        for fee in fees:
            fee_records.append({
                'subject': row['subject'],
                'title': row['title'],
                'term_name': row['term_name'],
                'term_id': row['term_id'],
                'fee_type': fee.get('description', ''),
                'amount': float(fee.get('amount', 0))
            })
    except (json.JSONDecodeError, TypeError):
        pass

df_fees = pd.DataFrame(fee_records)
print(f"Total fee entries: {len(df_fees)}")
print(f"Sections with fees: {df_fees_raw.shape[0]}")
print(f"\nFee types:")
print(df_fees['fee_type'].value_counts().head(10).to_string())

# Fee amount distribution by type
if len(df_fees) > 0:
    fig = px.box(
        df_fees, x='fee_type', y='amount',
        title='Fee Amount Distribution by Type',
        labels={'amount': 'Amount (AED)', 'fee_type': 'Fee Type'},
        color='fee_type'
    )
    fig.update_layout(template=TEMPLATE, height=450, showlegend=False)
    fig.show()

In [ ]:
# Fee trends over time
if len(df_fees) > 0:
    df_fee_trend = df_fees.groupby(['term_id', 'term_name', 'fee_type']).agg(
        avg_amount=('amount', 'mean'),
        count=('amount', 'count')
    ).reset_index().sort_values('term_id')
    
    # Get top fee types
    top_fees = df_fees['fee_type'].value_counts().head(5).index.tolist()
    df_fee_trend_top = df_fee_trend[df_fee_trend['fee_type'].isin(top_fees)]
    
    fig = px.line(
        df_fee_trend_top, x='term_name', y='avg_amount', color='fee_type',
        title='Average Fee Amount Over Time by Type',
        labels={'avg_amount': 'Amount (AED)', 'term_name': 'Semester'},
        markers=True
    )
    fig.update_layout(template=TEMPLATE, height=450, xaxis=dict(tickangle=-45, dtick=4))
    fig.show()

---
## 10. Course Description Text Analysis

In [ ]:
# Analyze course descriptions
df_desc = pd.read_sql_query("""
    SELECT subject, course_number, description, department
    FROM catalog
    WHERE description != '' AND LENGTH(description) > 20
""", conn)

# Description length distribution
df_desc['desc_length'] = df_desc['description'].str.len()
df_desc['word_count'] = df_desc['description'].str.split().str.len()

fig = px.histogram(
    df_desc, x='word_count', nbins=50,
    title='Course Description Length Distribution (Word Count)',
    labels={'word_count': 'Words', 'count': 'Courses'},
    color_discrete_sequence=[AUS_GOLD]
)
fig.update_layout(template=TEMPLATE, height=400)
fig.show()

print(f"Average description: {df_desc['word_count'].mean():.0f} words")
print(f"Longest: {df_desc.loc[df_desc['word_count'].idxmax(), 'subject']} {df_desc.loc[df_desc['word_count'].idxmax(), 'course_number']} ({df_desc['word_count'].max()} words)")
print(f"Shortest: {df_desc.loc[df_desc['word_count'].idxmin(), 'subject']} {df_desc.loc[df_desc['word_count'].idxmin(), 'course_number']} ({df_desc['word_count'].min()} words)")

In [ ]:
# Most common words in course descriptions (excluding stop words)
stop_words = {'the', 'and', 'of', 'to', 'in', 'a', 'is', 'for', 'on', 'with', 'an', 
              'are', 'as', 'be', 'by', 'from', 'or', 'at', 'this', 'that', 'it',
              'will', 'their', 'its', 'also', 'such', 'may', 'can', 'which', 'has',
              'not', 'all', 'but', 'have', 'been', 'other', 'into', 'more', 'both',
              'including', 'through', 'between', 'about', 'these', 'each', 'how',
              'various', 'well', 'use', 'used', 'using', 'they', 'than', 'topics',
              'course', 'students', 'study', 'introduction', 'methods'}

all_words = []
for desc in df_desc['description']:
    words = re.findall(r'[a-z]+', desc.lower())
    all_words.extend(w for w in words if w not in stop_words and len(w) > 3)

word_counts = Counter(all_words).most_common(40)
df_words = pd.DataFrame(word_counts, columns=['word', 'count'])

fig = px.bar(
    df_words, x='count', y='word', orientation='h',
    title='Most Common Words in Course Descriptions (Top 40)',
    labels={'count': 'Frequency', 'word': ''},
    color='count', color_continuous_scale='Viridis'
)
fig.update_layout(template=TEMPLATE, height=700, yaxis=dict(autorange='reversed'))
fig.update_coloraxes(showscale=False)
fig.show()

---
## 11. Corequisite Analysis

In [ ]:
# Corequisite pairs
df_coreqs = pd.read_sql_query("""
    SELECT DISTINCT
        c.subject || ' ' || c.course_number as course,
        d.subject || ' ' || d.course_number as corequisite,
        c.subject as course_dept
    FROM course_dependencies d
    JOIN courses c ON c.crn = d.crn AND c.term_id = d.term_id
    WHERE d.dep_type = 'corequisite'
""", conn)

print(f"Unique corequisite pairs: {len(df_coreqs)}")

# Most common corequisite relationships
coreq_pairs = df_coreqs.groupby(['course', 'corequisite']).size().reset_index(name='count')
coreq_pairs = coreq_pairs.sort_values('count', ascending=False).head(20)

print("\nMost common corequisite links:")
for _, row in coreq_pairs.iterrows():
    print(f"  {row['course']} ↔ {row['corequisite']}")

# Departments with most corequisites
dept_coreq = df_coreqs.groupby('course_dept').size().sort_values(ascending=False).head(15)

fig = px.bar(
    x=dept_coreq.index, y=dept_coreq.values,
    title='Departments with Most Corequisite Requirements',
    labels={'x': 'Subject', 'y': 'Corequisite Pairs'},
    color=dept_coreq.values, color_continuous_scale='Purples'
)
fig.update_layout(template=TEMPLATE, height=400)
fig.update_coloraxes(showscale=False)
fig.show()

---
## 12. Schedule Type and Instructional Method Trends

In [ ]:
# Schedule types
df_sched_type = pd.read_sql_query("""
    SELECT schedule_type, COUNT(*) as count
    FROM courses
    WHERE schedule_type != ''
    GROUP BY schedule_type
    ORDER BY count DESC
""", conn)

fig = px.pie(
    df_sched_type.head(10), values='count', names='schedule_type',
    title='Course Schedule Types',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(template=TEMPLATE, height=450)
fig.show()

In [ ]:
# Instructional method evolution (especially online/hybrid after COVID)
df_method = pd.read_sql_query("""
    SELECT s.term_name, s.term_id,
           c.instructional_method,
           COUNT(*) as count
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE c.instructional_method != ''
    AND (s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%')
    GROUP BY s.term_id, c.instructional_method
    ORDER BY s.term_id
""", conn)

if len(df_method) > 0 and df_method['instructional_method'].nunique() > 1:
    fig = px.area(
        df_method, x='term_name', y='count', color='instructional_method',
        title='Instructional Methods Over Time (Showing COVID Impact)',
        labels={'count': 'Sections', 'term_name': 'Semester', 'instructional_method': 'Method'},
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig.update_layout(template=TEMPLATE, height=450, xaxis=dict(tickangle=-45, dtick=2))
    fig.show()
else:
    print(f"Instructional methods: {df_method['instructional_method'].unique()}")
    print("All sections use the same method or field is empty.")

---
## 13. Lab vs Lecture Distribution

In [ ]:
# Lab sections analysis
df_lab = pd.read_sql_query("""
    SELECT s.term_name, s.term_id,
           SUM(CASE WHEN c.is_lab = 1 THEN 1 ELSE 0 END) as lab_sections,
           SUM(CASE WHEN c.is_lab = 0 THEN 1 ELSE 0 END) as lecture_sections,
           COUNT(*) as total
    FROM courses c
    JOIN semesters s ON c.term_id = s.term_id
    WHERE s.term_name LIKE 'Fall%' OR s.term_name LIKE 'Spring%'
    GROUP BY c.term_id
    ORDER BY c.term_id
""", conn)

df_lab['lab_pct'] = df_lab['lab_sections'] / df_lab['total'] * 100

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
    x=df_lab['term_name'], y=df_lab['lecture_sections'],
    name='Lectures', marker_color='#3498db'
), secondary_y=False)

fig.add_trace(go.Bar(
    x=df_lab['term_name'], y=df_lab['lab_sections'],
    name='Labs', marker_color='#e74c3c'
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_lab['term_name'], y=df_lab['lab_pct'],
    name='Lab %', mode='lines+markers',
    line=dict(color='#2ecc71', width=2), marker=dict(size=4)
), secondary_y=True)

fig.update_layout(
    template=TEMPLATE,
    title='Lecture vs Lab Sections Over Time',
    barmode='stack',
    height=450,
    xaxis=dict(tickangle=-45, dtick=4),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.update_yaxes(title_text='Sections', secondary_y=False)
fig.update_yaxes(title_text='Lab %', secondary_y=True)
fig.show()

# Which subjects have the most labs?
df_lab_subj = pd.read_sql_query("""
    SELECT subject,
           SUM(CASE WHEN is_lab = 1 THEN 1 ELSE 0 END) as labs,
           COUNT(*) as total,
           ROUND(100.0 * SUM(CASE WHEN is_lab = 1 THEN 1 ELSE 0 END) / COUNT(*), 1) as lab_pct
    FROM courses
    GROUP BY subject
    HAVING total >= 50
    ORDER BY lab_pct DESC
    LIMIT 20
""", conn)

print("\nSubjects with highest lab percentage:")
for _, row in df_lab_subj.head(10).iterrows():
    print(f"  {row['subject']:6s} {row['lab_pct']:5.1f}% labs ({row['labs']}/{row['total']})")

---
## 14. Cross-Department Prerequisites

Which departments depend on each other the most?

In [ ]:
# Cross-department prerequisite matrix
df_cross = pd.read_sql_query("""
    SELECT DISTINCT c.subject as course_dept, d.subject as prereq_dept,
           COUNT(DISTINCT c.subject || c.course_number) as course_count
    FROM course_dependencies d
    JOIN courses c ON c.crn = d.crn AND c.term_id = d.term_id
    WHERE d.dep_type = 'prerequisite' AND c.subject != d.subject
    GROUP BY c.subject, d.subject
    HAVING course_count >= 3
    ORDER BY course_count DESC
""", conn)

# Create a matrix
top_depts = list(set(df_cross.head(50)['course_dept'].tolist() + df_cross.head(50)['prereq_dept'].tolist()))
df_matrix = df_cross.pivot_table(index='course_dept', columns='prereq_dept', 
                                  values='course_count', fill_value=0)

# Filter to interesting departments
common_depts = df_matrix.sum(axis=1).sort_values(ascending=False).head(20).index.tolist()
common_prereqs = df_matrix.sum(axis=0).sort_values(ascending=False).head(20).index.tolist()
all_relevant = sorted(set(common_depts + common_prereqs))
df_matrix_filtered = df_matrix.reindex(index=all_relevant, columns=all_relevant, fill_value=0)
# Remove empty rows/cols
df_matrix_filtered = df_matrix_filtered.loc[(df_matrix_filtered.sum(axis=1) > 0), (df_matrix_filtered.sum(axis=0) > 0)]

fig = px.imshow(
    df_matrix_filtered,
    color_continuous_scale='Blues',
    title='Cross-Department Prerequisite Dependencies',
    labels={'x': 'Prerequisite Department', 'y': 'Course Department', 'color': 'Courses'},
    aspect='auto'
)
fig.update_layout(template=TEMPLATE, height=600)
fig.show()

print("Top cross-department dependencies:")
for _, row in df_cross.head(15).iterrows():
    print(f"  {row['course_dept']} depends on {row['prereq_dept']}: {row['course_count']} courses")

---
## 15. Summary Statistics

In [ ]:
# Final summary
stats = {}
for table in ['semesters', 'subjects', 'courses', 'instructors', 'levels', 
              'attributes', 'catalog', 'section_details', 'course_dependencies']:
    stats[table] = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]

print("=" * 60)
print("AUS Course Database — Summary Statistics")
print("=" * 60)
print(f"")
print(f"  Time span:           2005 – 2026 ({stats['semesters']} semesters)")
print(f"  Total sections:      {stats['courses']:,}")
print(f"  Unique courses:      3,473")
print(f"  Subject areas:       {stats['subjects']}")
print(f"  Instructors:         {stats['instructors']:,}")
print(f"  Academic levels:     {stats['levels']}")
print(f"  Course attributes:   {stats['attributes']}")
print(f"  Catalog entries:     {stats['catalog']:,}")
print(f"  Section details:     {stats['section_details']:,}")
print(f"  Dependencies:        {stats['course_dependencies']:,}")
print(f"")
print(f"  Database generated by AUSCrawl")
print(f"  https://github.com/DeadPackets/AUSCrawl")
print("=" * 60)

conn.close()